In [1]:
import pandas as pd
import glob
import re
import time
import os
from selenium import webdriver
from datetime import datetime
from random import randint
from PyPDF2 import PdfFileReader
from sklearn import linear_model


from library import Dataset
from Database import update_sentiment
from Crawler.outlook import outlook_initialize, outlook_get_marquee_link
from Crawler.marquee import gs_save_page
from Crawler.settings import REPORT_TYPE_MAPPING
from uti import UrlManager, Logger, By, DataLoader

UM = UrlManager()
logger = Logger()
DL = DataLoader()

Current root directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Current Code directory: C:/Users/JayYang/Platform/Macquarie
Current Database directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/Macquarie
Current working directory C:\Users\JayYang\Platform\Macquarie


2022-06-10 12:08:04,374 P[10396] [MainThread 24692] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2022-06-10 12:08:04,375 P[10396] [MainThread 24692] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-06-10 12:08:04,375 P[10396] [MainThread 24692] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-06-10 12:08:04,376 P[10396] [MainThread 24692] Port number was not identified, cannot send any request


Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0
Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0


In [2]:
options = webdriver.ChromeOptions()
download_path = DL.database_path
download_path = download_path.replace('/', '\\')
profile = {# "plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
         "download.default_directory": os.path.join(download_path, "Reports"),  # Path needs to be in \\ instead of /
          "download.extensions_to_open": "applications/pdf",
          "plugins.always_open_pdf_externally": True,
            "download.prompt_for_download": False
         
}
options.add_experimental_option("prefs", profile)

browser = UM.start_browser(chrome_options=options)
UM.get_url(browser, "https://www.macquarieinsights.com/#/login")
print("Please enter tsang@a-s-capital.com, and inputg the PIN from email.")



2022-05-27 07:08:35,517 P[39152] [MainThread 38948] 

====== WebDriver manager ======
2022-05-27 07:08:35,517 P[39152] [MainThread 38948] ====== WebDriver manager ======
Current google-chrome version is 102.0.5005
2022-05-27 07:08:35,599 P[39152] [MainThread 38948] Current google-chrome version is 102.0.5005
Get LATEST driver version for 102.0.5005
2022-05-27 07:08:35,600 P[39152] [MainThread 38948] Get LATEST driver version for 102.0.5005
There is no [win32] chromedriver for browser 102.0.5005 in cache
2022-05-27 07:08:35,639 P[39152] [MainThread 38948] There is no [win32] chromedriver for browser 102.0.5005 in cache
Get LATEST driver version for 102.0.5005
2022-05-27 07:08:35,640 P[39152] [MainThread 38948] Get LATEST driver version for 102.0.5005
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
2022-05-27 07:08:35,671 P[39152] [MainThread 38948] Trying to download new driver from https://chromedriver.storage.google

07:08:35 E:/jayyang/OneDrive - Alpha Sherpa Capital/Database/Macquarie\Log/20220527 created.
07:08:35 Starting Browser


Driver has been saved in cache [C:\Users\amazn\.wdm\drivers\chromedriver\win32\102.0.5005.61]
2022-05-27 07:08:35,838 P[39152] [MainThread 38948] Driver has been saved in cache [C:\Users\amazn\.wdm\drivers\chromedriver\win32\102.0.5005.61]


07:08:37: Accessing url... 07:08:40 Success. Idle for 5 seconds...
Please enter tsang@a-s-capital.com, and inputg the PIN from email.


In [3]:
while True:
    try:
        button = browser.find_element_by_link_text('View more')
        button.click()
        time.sleep(0.5)
    except:
        break

In [2]:
elems = browser.find_elements_by_xpath("//a[@href]")
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))
useful_links = [link for link in links if link.find('researchId') != -1]

df = pd.DataFrame(useful_links)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

for i in range(0,len(df)):
    browser.get(df.iloc[i,0])
    time.sleep(3)
    button = browser.find_element_by_link_text('DOWNLOAD')
    button.click()
    
    time.sleep(0.5)

NameError: name 'browser' is not defined

## Download

In [7]:
df = DL.loadDB('balabala.csv')

12:12:31 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/Macquarie\balabala.csv


In [11]:
path = f"{DL.database_path}/Reports/*.pdf"
# path = f"{DL.database_path}/OldReports/*.pdf"
files=glob.glob(path)
df = pd.DataFrame(files, columns =['Path'])
# df['report_id'] = df['Path']

## Extract text from files page 0

In [12]:
df['Text'] = ''
for i in range(0,len(df)):
    path = df.loc[i,'Path']
    pdf=PdfFileReader(path)
    text=pdf.getPage(0).extractText()
    txt = text.replace('\n','')
    df.loc[i,'Text'] = txt
df[df['Text']=='']

,Path,Text


In [13]:
df

,Path,Text
0,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 3 for i mpor ta nt ...
1,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...
2,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 5 for i mpor ta nt ...
3,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...
4,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 2 for i mpor ta nt ...


## Cleaning: time and date

In [14]:
for i in range(0,len(df)):
    try:
        path = df.loc[i,'Path']
        pdf=PdfFileReader(path)
        text=pdf.getPage(-1).extractText()
        txt = text.replace('\n','')
        df.loc[i,'publish_date_and_time'] = txt
    except:
        print(i)
    time = df.loc[i,'publish_date_and_time']
    time= time.replace('This publication was disseminated on ','')
    time = time.replace('at ','')
    time = time.replace('.','')
    time= time.replace(' UTC','')
    df.loc[i,'publish_date_and_time'] = time
df[df['publish_date_and_time']=='']

,Path,Text,publish_date_and_time


In [15]:
df

,Path,Text,publish_date_and_time
0,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 3 for i mpor ta nt ...,09 June 2022 16:48
1,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 16:36
2,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 5 for i mpor ta nt ...,09 June 2022 16:38
3,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 23:50
4,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 2 for i mpor ta nt ...,08 June 2022 16:50


In [16]:
#Analyst
df['Primary Analyst'] = ''

for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    #primary analyst
    txt_lst = txt.split()
    emails = [i for i in txt_lst if '@macquarie.com' in i]
    emails = [i.replace('@macquarie.com', '') for i in emails]
    emails = [i.split('.') for i in emails]
    if len(emails) > 0:
        analyst_pri = [i.capitalize() for i in emails[0]]
        analyst_pri = ' '.join(analyst_pri)
        df.loc[x,'Primary Analyst'] = analyst_pri
df['Primary Analyst'].unique()

array([''], dtype=object)

In [17]:
df

,Path,Text,publish_date_and_time,Primary Analyst
0,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 3 for i mpor ta nt ...,09 June 2022 16:48,
1,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 16:36,
2,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 5 for i mpor ta nt ...,09 June 2022 16:38,
3,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 23:50,
4,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 2 for i mpor ta nt ...,08 June 2022 16:50,


In [12]:
import pdb 

In [21]:
df['ticker'] = ''
for i in range(0,len(df)):
    txt = df.loc[i,'Text']
    namere=re.compile(r'@macquarie.com\s+([A-ZÉa-z0-9&-( ]+)\)\s')
    name=namere.findall(txt)
    ticker = ''
    company_name = ''
    if len(name) >0:
        if len(name) >1:
            name = name[-1]
        else:
            name = name[0]
        name = name.split(' ')
        tick = ' '.join([name[-2],name[-1]])
        ticker = tick.replace('(','')
        ticker = ticker.replace(')','')
        company_name = name[:len(name)-2]
        company_name = ' '.join(company_name)
    df.loc[i,'Company Name'] = company_name
    df.loc[i,'ticker'] = ticker
    if df.loc[i,'ticker'] == '':
        tickre = re.compile(r'\d{3,4}\s(?:HK|JP|TT)')
        ticker = tickre.findall(txt)
        print(ticker)
        if len(ticker)>0:
            ticker = ticker[0]
        else:
            ticker = ''
    df.loc[i,'ticker'] = ticker
df[df['ticker']=='']
df['ticker'].unique()
# df.loc[1, 'ticker'] = '2333 HK'
# df.loc[3, 'ticker'] = '9868 HK'

['9988 HK']
['1929 HK']
['9618 HK']
['2382 HK', '2382 HK']
['8035 JP', '8035 JP']


array(['9988 HK', '1929 HK', '9618 HK', '2382 HK', '8035 JP'],
      dtype=object)

In [19]:
txt

'  P leas e r efer to page 2 for  i mpor ta nt discl osures an d analyst cer tific atio n, or on our  w ebsite w w w .macquar ie.com/r esea r ch/disclosures.    Fl ashnote  9J une 2022 J apan EQUITI ES   8 0 3 5 J P Ou tp e rfo rm P ric e ( a t CLOSE# , 0 8  Ju n  20 2 2) ¥5 8 ,80 0  V aluat ion ¥  80, 000  - P ric e t o B o ok 12-mont h t arget ¥  80, 000 Ups ide/ Downs ide %  +36. 1 12-mont h TS R %  +39. 2 V olat ility I ndex Medium GI CS s ec t or S emic onduct ors & S emic onduc t orE quipment Mark et c ap ¥bn  9, 243 Mark et c ap US $m  71, 091 30-day  av g t urnov er US $m  463. 7 Number s hares on iss ue m  157. 2  I nvestm ent fundam ental s Ye a r  e n d  31  M ar  2022A 2023E 2024E 2025E Revenue bn 2,003.8 2,400.2 2,591.1 2,738.4 EBIT bn 599.3 735.4 810.0 850.2 EBIT growth % 86.9 22.7 10.1 5.0 Reported profit bn 437.1 549.7 605.4 635.5 EPS rep ¥ 2,801 3,523 3,911 4,190 EPS rep growth % 81.5 25.8 11.0 7.1 EPS adj ¥ 2,833 3,523 3,911 4,190 EPS adj growth % 79.8 24.3 11.0 7.1 P

In [10]:
df

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name
0,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 3 for i mpor ta nt ...,09 June 2022 16:48,,9988 HK,
1,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 16:36,,1929 HK,
2,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 5 for i mpor ta nt ...,09 June 2022 16:38,,9618 HK,
3,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 6 for i mpor ta nt ...,09 June 2022 23:50,,2382 HK,
4,C:/Users/JayYang/OneDrive - Alpha Sherpa Capit...,P leas e r efer to page 2 for i mpor ta nt ...,08 June 2022 16:50,,JP,


In [15]:
#Current Rating
df['Current Rating'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    r = re.compile('[JP|HK|TT]\s+([A-Za-z]+)\sPrice\s')
    rating_curr = r.findall(txt)
    if len(rating_curr)>0:
        rating_curr = rating_curr[0]
    else:
        rating_curr = ''
    df.loc[x,'Current Rating'] = rating_curr
df[df['Current Rating']=='']

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating


In [16]:
df

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating
0,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 6 for important disclos...,26 May 2022 22:48,Ellie Jiang,9988 HK,,Outperform
1,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 7 for important disclos...,26 May 2022 11:25,Jeffrey Ohlweiler,5274 TT,ASPEED Technology,Outperform
2,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 12 for important disclo...,26 May 2022 18:01,Ellie Jiang,9618 HK,,Outperform
3,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 9 for important disclos...,26 May 2022 00:09,Damian Thong,6951 JP,JEOL,Outperform
4,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 6 for important disclos...,26 May 2022 06:00,Shentao Tang,3038 JP,Kobe Bussan,Outperform
5,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 2 for important disclos...,26 May 2022 09:55,Damian Thong,7731 JP,Nikon,Outperform
6,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 2 for important disclos...,26 May 2022 02:02,Damian Thong,6758 JP,Sony,Neutral


In [17]:
# Headline Summary
df['headline'] = ''
df['summary'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    smr = re.compile(r'(?s:.*@macquarie.com)\s+(.*) This publication was downloaded')
    smr = smr.findall(txt)
    if len(smr) > 0:
        if len(smr) == 1:
            smr = smr[0]
        else:
            smr = smr[-1]
    else:
        smr = ''
    smr_list = smr.split('  ')
    headline = smr_list[0]
    summary = ' '.join(smr_list[1:])
    df.loc[x, 'headline'] = headline
    df.loc[x,'summary'] = summary
df[['headline', 'summary']]

,headline,summary
0,Alibaba Group Holding (BABA US/9988 HK) 4QFY3/22,earnings tracking better than expected Key poi...
1,ASPEED Technology (5274 TT) The final piece of...,We upgrade to OP from Neutral after the 46% pu...
2,JD.com (JD US/9618 HK) The best pick emerging ...,JD is a top gainer on easing COVID-19 logistic...
3,JEOL (6951 JP) We see JEOL beating guidance an...,Key points stance. We affirm our Outperform ra...
4,Kobe Bussan (3038 JP) Poised to win the race a...,"April numbers came in robustly, with GPM bounc..."
5,Nikon (7731 JP) Much to like in the IR Day; St...,N in our view better than we had expected. We ...
6,Sony (6758 JP) Strategic pivot in games is now...,and we like it Strategic pivot is now fully cl...


In [18]:
#GICS
df['gics1'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    gics = re.compile(r'GICS\ssector\s+([A-Za-z-()/,& ]+)\s+Mar')
    gics1 = gics.findall(txt)
    if len(gics1) > 0:
        gics1 = gics1[0]
    else:
        gics1 = ''
    df.loc[x,'gics1'] = gics1
df[df['gics1']=='']   

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary,gics1


In [19]:
#Report type
df['Report Type'] = 'Ad hoc'
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    er = re.compile(r'(Strong|strong|Good|good)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+results')
    er = er.findall(txt)
    if len(er)>0:
        df.loc[x,'Report Type'] = 'Earning Review'
    er = re.compile(r'(Beat|beat)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+consensus')
    er = er.findall(txt)
    if len(er)>0:
        df.loc[x,'Report Type'] = 'Earning Review'
    
    if df.loc[x,'Report Type'] != 'Earning Review':
        txt = df.loc[x,'headline']
        substrings = ['3Q','1Q','2Q','4Q','1H','2H','FY','2018','2019','2020','2021','earnings','Earnings']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Earning Related'
        else:
            txt = df.loc[x,'Text']
            substrings = ['Initiate','initiate coverage','Initiating','initiating']
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Initiation'
            else: 
                substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from','Upgrading','upgrading'] 
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Upgrade'
                else:
                    substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'Rating Downgrade'   
                    else:
                        substrings = ['Earnings and target price revision']
                        if any(substring in txt for substring in substrings)== True:
                            df.loc[x,'Report Type'] = 'TP Related and Estimate Change'

    if df.loc[x,'Report Type'] == 'Ad hoc':
        txt = df.loc[x,'Text']
        substrings = ['Initiate','initiate','Initiating','initiating']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Initiation'
        else: 
            substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from'] 
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Rating Upgrade'
            else:
                substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Downgrade'   
                else:
                    substrings = ['Earnings and target price revision']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'TP Related and Estimate Change'
                        
    
    if df.loc[x,'Report Type'] == 'Earning Related':
        txt = df.loc[x,'summary']
        substrings = ['beat','miss','in line','above','below','In line','Above','Below','Beat','Miss','remains','remain','beats','misses','Beats','Misses','earnings','Earnings''results were better']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Earning Review'
            
        else:
            txt = df.loc[x,'Text']
            substrings = ['Initiate','initiate','Initiating','initiating']
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Initiation'
            else: 
                substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from'] 
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Upgrade'
                else:
                    substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'Rating Downgrade'   
                    else:
                        substrings = ['overshot the consensus']
                        if any(substring in txt for substring in substrings)== True:
                            df.loc[x,'Report Type'] = 'Earning Review'  
                        else:
                            df.loc[x,'Report Type'] = 'Ad hoc'
                            
    if df.loc[x,'Report Type'] == 'TP Related and Estimate Change':
        txt = df.loc[x,'Text']
        tp = re.compile(r'(Earnings and target price revision)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+')
        tp = tp.findall(txt)[0]
        substrings1 = ['TP','Target Price','target price','Target price']
        substrings2 = ['unchanged target price']
        if any(substring in tp for substring in substrings1)== True and any(substring in tp for substring in substrings2)== False:
            df.loc[x,'Report Type'] = 'TP change'
        else:
            df.loc[x,'Report Type'] = 'Estimate change'
        substrings = ['No change']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] ='Ad hoc'
            
            
                            
df[df['Report Type']=='']

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary,gics1,Report Type


In [20]:
data = Dataset(df)
data.df['headline_senti'], _ = update_sentiment(data.df['headline'])
data.df['summary_senti'], _ = update_sentiment(data.df['summary'])
data.clean(tz_name='Europe/London', mode='stable')
df = data.df

2022-05-27 07:17:53.803779


Device: cuda
'update_sentiment'  23179.10 ms
2022-05-27 07:18:16.809227
Device: cuda
'update_sentiment'  1348.57 ms


In [21]:
df = data.df
df

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary,gics1,Report Type,headline_senti,summary_senti,exch_location,exch_region,date_local,time_local,release_period,ticker_updated
0,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 6 for important disclos...,26 May 2022 22:48,Ellie Jiang,9988 HK,,Outperform,Alibaba Group Holding (BABA US/9988 HK) 4QFY3/22,earnings tracking better than expected Key poi...,Retailing,Earning Review,neutral,positive,Hong Kong,Asia,2022-05-27,05:48:00,Before,9988 HK
1,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 7 for important disclos...,26 May 2022 11:25,Jeffrey Ohlweiler,5274 TT,ASPEED Technology,Outperform,ASPEED Technology (5274 TT) The final piece of...,We upgrade to OP from Neutral after the 46% pu...,Semiconductors & Semiconductor Equipment,Rating Upgrade,neutral,positive,Taiwan,Asia,2022-05-26,18:25:00,After,5274 TT
2,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 12 for important disclo...,26 May 2022 18:01,Ellie Jiang,9618 HK,,Outperform,JD.com (JD US/9618 HK) The best pick emerging ...,JD is a top gainer on easing COVID-19 logistic...,Retailing,Ad hoc,positive,positive,Hong Kong,Asia,2022-05-27,01:01:00,Before,9618 HK
3,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 9 for important disclos...,26 May 2022 00:09,Damian Thong,6951 JP,JEOL,Outperform,JEOL (6951 JP) We see JEOL beating guidance an...,Key points stance. We affirm our Outperform ra...,Health Care Equipment & Services,TP change,positive,positive,Japan,Asia,2022-05-26,08:09:00,Before,6951 JP
4,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 6 for important disclos...,26 May 2022 06:00,Shentao Tang,3038 JP,Kobe Bussan,Outperform,Kobe Bussan (3038 JP) Poised to win the race a...,"April numbers came in robustly, with GPM bounc...",Food & Staples Retailing,TP change,positive,positive,Japan,Asia,2022-05-26,14:00:00,Within,3038 JP
5,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 2 for important disclos...,26 May 2022 09:55,Damian Thong,7731 JP,Nikon,Outperform,Nikon (7731 JP) Much to like in the IR Day; St...,N in our view better than we had expected. We ...,Consumer Durables & Apparel,Ad hoc,positive,positive,Japan,Asia,2022-05-26,17:55:00,After,7731 JP
6,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,Please refer to page 2 for important disclos...,26 May 2022 02:02,Damian Thong,6758 JP,Sony,Neutral,Sony (6758 JP) Strategic pivot in games is now...,and we like it Strategic pivot is now fully cl...,Consumer Durables & Apparel,Ad hoc,neutral,positive,Japan,Asia,2022-05-26,10:02:00,Within,6758 JP


In [22]:
df1 = pd.read_csv(f'{DL.database_path}/scoring.csv')

In [23]:
df1

,report_id,Text,publish_date_and_time,Primary Analyst,Secondary Analysts,Current Rating,headline,Target Price,TSR,Free Float,...,w5,d0_r_af,d1_r_af,d2_r_af,d3_r_af,d4_r_af,d5_r_af,Type,w6,w7
0,C:\Users\ZeyuWang\Desktop\Macquarie New\HK\rep...,Please refer to page 5 for important disclos...,01 June 2018 04:10,Scott Russell,Scott Russell;Amy Zhou,Neutral,AIA Group HK bottoming out in 1Q18 Key points...,66.00,-6.5,NaN,...,1.0,0.082375,0.027879,-0.544328,-0.898551,0.082375,-0.162856,Earning Review-Neutral,0.084541,0.143021
1,C:\Users\ZeyuWang\Desktop\Macquarie New\JP\rep...,Please refer to page 2 for important disclos...,04 June 2018 01:13,Keisuke Moriyama,Keisuke Moriyama,Outperform,Dynamic at work of more bad news accelerating ...,"2,000",61.3,62.0,...,1.0,-0.502320,-1.036390,-1.036390,-1.036390,-1.036390,-1.036390,Ad hoc-Outperform,0.084541,0.057353
2,C:\Users\ZeyuWang\Desktop\Macquarie New\KS\rep...,Please refer to page 2 for important disclos...,04 June 2018 05:02,Kwang Cho,Kwang Cho,Outperform,Big win in the large enterprise market,"65,000",48.8,NaN,...,1.0,-0.316981,0.898952,1.066667,1.150524,0.731237,0.773166,Ad hoc-Outperform,0.084541,0.057353
3,C:\Users\ZeyuWang\Desktop\Macquarie New\TT\rep...,Please refer to page 2 for important disclos...,05 June 2018 08:42,Corinne Jian,Corinne Jian;Shannen Lin;Steven Yang,Underperform,Combined April-May revenue up 12% YoY,90,-30.4,88.0,...,0.0,-0.658372,-1.193256,-1.193256,-1.193256,-1.193256,-1.193256,Ad hoc-Underperform,0.098515,0.044243
4,C:\Users\ZeyuWang\Desktop\Macquarie New\JP\rep...,Please refer to page 2 for important disclos...,05 June 2018 11:26,Janet Lewis,Janet Lewis;Shentao Tang,Neutral,More fuel economy testing problems in Japan le...,"3,750",13.4,72.0,...,1.0,0.257145,-1.050322,-1.050322,-1.050322,-1.050322,-1.050322,Ad hoc-Neutral,0.098515,0.081888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6813,C:\Users\ZeyuWang\Desktop\Macquarie New\JP\rep...,Please refer to page 8 for important disclos...,25 January 2022 18:52,Damian Thong,Damian Thong;Aven Guan;Wendy Pan,Outperform,3Q beat; FY guidance looks conservative,"46,500",51.7,NaN,...,1.0,0.382420,-0.211187,-1.028539,-1.028539,-1.028539,-1.028539,Earning Review-Outperform,0.161230,0.093250
6814,C:\Users\ZeyuWang\Desktop\Macquarie New\KS\rep...,Please refer to page 6 for important disclos...,25 January 2022 15:48,James Hong,James Hong;Jaeseo Lee,Outperform,Resilient GM; strong product mix gain,"260,000",33.0,68.0,...,1.0,-0.863494,-1.140602,-1.140602,-1.140602,-1.140602,-1.140602,TP change-Outperform,0.161230,0.091141
6815,C:\Users\ZeyuWang\Desktop\Macquarie New\TT\rep...,Please refer to page 9 for important disclos...,25 January 2022 23:25,Jeffrey Ohlweiler,Jeffrey Ohlweiler;Cindy Chiang,Outperform,Strong 4Q finish and positive 2022 outlook,66.1,75.1,99.0,...,1.0,0.921768,0.921768,0.921768,0.921768,0.921768,0.921768,Earning Review-Outperform,0.161230,0.093250
6816,C:\Users\ZeyuWang\Desktop\Macquarie New\KS\rep...,Please refer to page 6 for important disclos...,25 January 2022 16:49,Andy Kim,Andy Kim,Outperform,We expect a better shareholder return policy,"18,000",39.7,59.0,...,1.0,-0.356600,-1.156600,-1.156600,-1.156600,-1.156600,-1.156600,TP change-Outperform,0.161230,0.091141


In [24]:
df['side'] = ''
for x in range(len(df)):
    if df.loc[x,'summary_senti'] == 'positive' and df.loc[x,'headline_senti'] == 'positive':
        df.loc[x,'side'] = 'long'
    elif df.loc[x,'summary_senti'] == 'negative' or df.loc[x,'headline_senti'] == 'negative':
        df.loc[x,'side'] = 'short'
df = df.dropna(subset=['side'])
df = df.reset_index(drop=True)

In [25]:
for i in range(len(df)):
    if df.loc[i,'side'] == 'long':
        if df.loc[i,'Current Rating'] == 'Outperform':
            df.loc[i,'w5'] = 1
        else:
            df.loc[i,'w5'] = 0
    else:
        if df.loc[i,'Current Rating'] == 'Outperform':
            df.loc[i,'w5'] = 0
        else:
            df.loc[i,'w5'] = 1

In [26]:
df1 = pd.concat([df1,df])

In [27]:
df1

,report_id,Text,publish_date_and_time,Primary Analyst,Secondary Analysts,Current Rating,headline,Target Price,TSR,Free Float,...,d2_r_af,d3_r_af,d4_r_af,d5_r_af,Type,w6,w7,Path,ticker,Company Name
0,C:\Users\ZeyuWang\Desktop\Macquarie New\HK\rep...,Please refer to page 5 for important disclos...,01 June 2018 04:10,Scott Russell,Scott Russell;Amy Zhou,Neutral,AIA Group HK bottoming out in 1Q18 Key points...,66.00,-6.5,NaN,...,-0.544328,-0.898551,0.082375,-0.162856,Earning Review-Neutral,0.084541,0.143021,NaN,NaN,NaN
1,C:\Users\ZeyuWang\Desktop\Macquarie New\JP\rep...,Please refer to page 2 for important disclos...,04 June 2018 01:13,Keisuke Moriyama,Keisuke Moriyama,Outperform,Dynamic at work of more bad news accelerating ...,"2,000",61.3,62.0,...,-1.036390,-1.036390,-1.036390,-1.036390,Ad hoc-Outperform,0.084541,0.057353,NaN,NaN,NaN
2,C:\Users\ZeyuWang\Desktop\Macquarie New\KS\rep...,Please refer to page 2 for important disclos...,04 June 2018 05:02,Kwang Cho,Kwang Cho,Outperform,Big win in the large enterprise market,"65,000",48.8,NaN,...,1.066667,1.150524,0.731237,0.773166,Ad hoc-Outperform,0.084541,0.057353,NaN,NaN,NaN
3,C:\Users\ZeyuWang\Desktop\Macquarie New\TT\rep...,Please refer to page 2 for important disclos...,05 June 2018 08:42,Corinne Jian,Corinne Jian;Shannen Lin;Steven Yang,Underperform,Combined April-May revenue up 12% YoY,90,-30.4,88.0,...,-1.193256,-1.193256,-1.193256,-1.193256,Ad hoc-Underperform,0.098515,0.044243,NaN,NaN,NaN
4,C:\Users\ZeyuWang\Desktop\Macquarie New\JP\rep...,Please refer to page 2 for important disclos...,05 June 2018 11:26,Janet Lewis,Janet Lewis;Shentao Tang,Neutral,More fuel economy testing problems in Japan le...,"3,750",13.4,72.0,...,-1.050322,-1.050322,-1.050322,-1.050322,Ad hoc-Neutral,0.098515,0.081888,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,NaN,Please refer to page 12 for important disclo...,26 May 2022 18:01,Ellie Jiang,NaN,Outperform,JD.com (JD US/9618 HK) The best pick emerging ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,9618 HK,
3,NaN,Please refer to page 9 for important disclos...,26 May 2022 00:09,Damian Thong,NaN,Outperform,JEOL (6951 JP) We see JEOL beating guidance an...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,6951 JP,JEOL
4,NaN,Please refer to page 6 for important disclos...,26 May 2022 06:00,Shentao Tang,NaN,Outperform,Kobe Bussan (3038 JP) Poised to win the race a...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,3038 JP,Kobe Bussan
5,NaN,Please refer to page 2 for important disclos...,26 May 2022 09:55,Damian Thong,NaN,Outperform,Nikon (7731 JP) Much to like in the IR Day; St...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:/jayyang/OneDrive - Alpha Sherpa Capital/Dat...,7731 JP,Nikon


In [28]:
dfh = df1[df1['exch_location']=='Japan'].reset_index(drop=True)
dft = df1[df1['exch_location']=='Taiwan'].reset_index(drop=True)
dfj = df1[df1['exch_location']=='Japan'].reset_index(drop=True)
# dfh = df1[df1['exch_location']=='Hong Kong'].reset_index(drop=True)

#   dft = df[df['Country']=='TT'].reset_index(drop=True)

#   dfk = df[df['Country']=='KS'].reset_index(drop=True)
# Hong Kong
# allset = [dfh]
# predset= [dfh[dfh['report_id'].isnull()==False]]  # , dfj[dfj['report_id'].isnull()==False] ,dft[dft['report_id'].isnull()==False]
# valset= [dfh[dfh['report_id'].isnull()==True]]  # , dfj[dfj['report_id'].isnull()==True] ,dft[dft['report_id'].isnull()==True]

# Taiwan
allset = [dft]
predset= [dft[dft['report_id'].isnull()==False]]  # , dfj[dfj['report_id'].isnull()==False] ,dft[dft['report_id'].isnull()==False]
valset= [dft[dft['report_id'].isnull()==True] ]  # , dfj[dfj['report_id'].isnull()==True] ,dft[dft['report_id'].isnull()==True]

# Japan
# allset = [dfj]
# predset= [dfj[dfj['report_id'].isnull()==False]]  # , dfj[dfj['report_id'].isnull()==False] ,dft[dft['report_id'].isnull()==False]
# valset= [dfj[dfj['report_id'].isnull()==True] ]  # , dfj[dfj['report_id'].isnull()==True] ,dft[dft['report_id'].isnull()==True]
vali_sets = []

for country in ['Japan', 'Taiwan', 'Hong Kong']:
    dft = df1[df1['exch_location']== country].reset_index(drop=True)
    allset = [dft]
    predset= [dft[dft['report_id'].isnull()==False]]  # , dfj[dfj['report_id'].isnull()==False] ,dft[dft['report_id'].isnull()==False]
    valset= [dft[dft['report_id'].isnull()==True] ]  # , dfj[dfj['report_id'].isnull()==True] ,dft[dft['report_id'].isnull()==True]


    for i in range(len(predset)):

        lst = list(allset[i]['side'].unique())
        w1 = []
        for x in lst:
            pf = predset[i][predset[i]['side']== x]
            num = pf['d0_r'].mean()
            w1.append(num)
        zip_iterator = zip(lst, w1)
        a_dictionary = dict(zip_iterator)
        for x in range(len(allset[i])):
            allset[i].loc[x,'w1'] = a_dictionary[allset[i].loc[x,'side']]


        lst = list(allset[i]['Primary Analyst'].unique())
        w2 = []
        for x in lst:
            pf = predset[i][predset[i]['Primary Analyst']== x]
            num = pf['d0_r'].mean()
            w2.append(num)
        zip_iterator = zip(lst, w2)
        a_dictionary = dict(zip_iterator)
        for x in range(len(allset[i])):
            allset[i].loc[x,'w2'] = a_dictionary[allset[i].loc[x,'Primary Analyst']]

        lst = list(allset[i]['Report Type'].unique())
        w3 = []
        for x in lst:
            pf = predset[i][predset[i]['Report Type']== x]
            num = pf['d0_r'].mean()
            w3.append(num)
        zip_iterator = zip(lst, w3)
        a_dictionary = dict(zip_iterator)
        for x in range(len(allset[i])):
            allset[i].loc[x,'w3'] = a_dictionary[allset[i].loc[x,'Report Type']]

        lst = list(allset[i]['gics1'].unique())
        w4 = []
        for x in lst:
            pf = predset[i][predset[i]['gics1']== x]
            num = pf['d0_r'].mean()
            w4.append(num)
        zip_iterator = zip(lst, w4)
        a_dictionary = dict(zip_iterator)
        for x in range(len(allset[i])):
            allset[i].loc[x,'w4'] = a_dictionary[allset[i].loc[x,'gics1']]


        lst = list(allset[i]['release_period'].unique())
        w6 = []
        for x in lst:
            pf = predset[i][predset[i]['release_period']== x]
            num = pf['d0_r'].mean()
            w6.append(num)
        zip_iterator = zip(lst, w6)
        a_dictionary = dict(zip_iterator)
        for x in range(len(allset[i])):
            allset[i].loc[x,'w6'] = a_dictionary[allset[i].loc[x,'release_period']]


        allset[i] = allset[i].fillna(value = { "w1": allset[i].loc[:,'w1'].mean(), "w2": allset[i].loc[:,'w2'].mean(), "w3": allset[i].loc[:,'w3'].mean(),
                                    "w4": allset[i].loc[:,'w4'].mean(), "w5": allset[i].loc[:,'w5'].mean(),"w6": allset[i].loc[:,'w6'].mean()})

        dd = allset[i]
        predset= [dd[dd['report_id'].isnull()==False],dd[dd['report_id'].isnull()==False]]
        valset= [dd[dd['report_id'].isnull()==True],dd[dd['report_id'].isnull()==True]]


        Y = predset[i].loc[:,'d0_r_af']
        X = predset[i].loc[:,['w1','w2','w3','w4','w5','w6']]
        regr = linear_model.LinearRegression()
        regr.fit(X, Y)
        try:
            predicted_r = regr.predict(valset[i].loc[:,['w1','w2','w3','w4','w5','w6']])
            sort_by = list(predicted_r)
            valset[i] = valset[i].reset_index(drop=True)
            for x in range(len(valset[i])):
                valset[i].loc[x,'sort_by'] = predicted_r[x]
            vali_sets.append(valset[i])
        except:
            print('No trade')
# vali_sets.append(dft)
val = pd.concat(vali_sets).reset_index(drop=True)
data= Dataset(val)
#trades = data._select_trades_with_portfolio_constraint(data.df, sort_by='sort_by',max_trade=2,max_skew=0).reset_index(drop=True)
#trades = trades[['ticker_updated','sort_by','side','headline_senti','summary_senti','Company Name','Primary Analyst','Report Type','gics1','release_period','headline','summary','publish_date_and_time']]
val = val[['ticker','date_local','release_period','side','gics1','Report Type','headline_senti','summary_senti','headline','summary','Current Rating',
           'Target Price','sort_by',
           'Primary Analyst','exch_location']].reset_index(drop=True)

In [29]:
val

,ticker,date_local,release_period,side,gics1,Report Type,headline_senti,summary_senti,headline,summary,Current Rating,Target Price,sort_by,Primary Analyst,exch_location
0,6951 JP,2022-05-26,Before,long,Health Care Equipment & Services,TP change,positive,positive,JEOL (6951 JP) We see JEOL beating guidance an...,Key points stance. We affirm our Outperform ra...,Outperform,NaN,0.143365,Damian Thong,Japan
1,3038 JP,2022-05-26,Within,long,Food & Staples Retailing,TP change,positive,positive,Kobe Bussan (3038 JP) Poised to win the race a...,"April numbers came in robustly, with GPM bounc...",Outperform,NaN,0.305467,Shentao Tang,Japan
2,7731 JP,2022-05-26,After,long,Consumer Durables & Apparel,Ad hoc,positive,positive,Nikon (7731 JP) Much to like in the IR Day; St...,N in our view better than we had expected. We ...,Outperform,NaN,0.088636,Damian Thong,Japan
3,6758 JP,2022-05-26,Within,,Consumer Durables & Apparel,Ad hoc,neutral,positive,Sony (6758 JP) Strategic pivot in games is now...,and we like it Strategic pivot is now fully cl...,Neutral,NaN,0.053665,Damian Thong,Japan
4,5274 TT,2022-05-26,After,,Semiconductors & Semiconductor Equipment,Rating Upgrade,neutral,positive,ASPEED Technology (5274 TT) The final piece of...,We upgrade to OP from Neutral after the 46% pu...,Outperform,NaN,-0.113978,Jeffrey Ohlweiler,Taiwan
5,9988 HK,2022-05-27,Before,,Retailing,Earning Review,neutral,positive,Alibaba Group Holding (BABA US/9988 HK) 4QFY3/22,earnings tracking better than expected Key poi...,Outperform,NaN,0.132533,Ellie Jiang,Hong Kong
6,9618 HK,2022-05-27,Before,long,Retailing,Ad hoc,positive,positive,JD.com (JD US/9618 HK) The best pick emerging ...,JD is a top gainer on easing COVID-19 logistic...,Outperform,NaN,0.014642,Ellie Jiang,Hong Kong


In [30]:
val = val[['ticker','date_local','release_period','side','gics1','Report Type','headline_senti','summary_senti','headline','summary','Current Rating',
           'Target Price','sort_by',
           'Primary Analyst','exch_location']].reset_index(drop=True)

In [31]:
val = val.rename(columns={"ticker": "Ticker", 
                          "date_local": "Date of publish",
                         "release_period": "Timing",
                         "side": "Side",
                         "sector": "gics1", 
                         "Report Type": "Report Type (Broker)",
                         "headline_senti": "Headline Sentiment",
                         "summary_senti": "Summary Sentiment",
                          "Primary Analyst": "Analyst Primary",
                         "headline": "Headline Content",
                         "summary": "Summary Content",
                         "Target Price": "Current Target Price", "sort_by": "Final Expectancy D0",
                         "long": "Top Analyst (long)",
                        "short": "Top Analyst (short)",
                        "analyst_pri": "Analyst Primary"
                         })

In [32]:
bench= pd.read_csv(f'{DL.database_path}/mac_full.csv')

In [33]:
for i in range(len(val)):
    needed = bench[(bench['headline_senti']==val.loc[i,'Headline Sentiment'])&(bench['summary_senti']==val.loc[i,'Summary Sentiment'])&(bench['Report Type']==val.loc[i,'Report Type (Broker)'])&(bench['exch_location']==val.loc[i,'exch_location'])]
    val.loc[i,'Blind Long expectancy'] = needed.dropna(subset=['l0'])['l0'].mean()
    val.loc[i,'Blind Short expectancy'] = needed.dropna(subset=['s0'])['s0'].mean()

In [34]:
anal = pd.read_csv(f'{DL.database_path}/mac_anl.csv')
val = val.merge(anal, on="Analyst Primary", how='left')
val[['Top Analyst (long)', 'Top Analyst (short)']] = val[['Top Analyst (long)', 'Top Analyst (short)']].fillna(False)
val['Broker']='Macquarie'

In [35]:
from datetime import datetime 

In [36]:
val.to_csv(f'{DL.database_path}/MAC'+datetime.today().strftime('%m%d')+'.csv',index = False)

In [37]:
datetime.today()

datetime.datetime(2022, 5, 27, 7, 18, 38, 505812)

In [38]:
results = DL.loadDB(f'{DL.database_path}/MAC'+datetime.today().strftime('%m%d')+'.csv')
print(results.columns)

results['rating_prev'] = ''
results['tp_prev'] = ''
results['tp_chg_pct'] = ''
results['up_or_down_side_pct'] = ''
results['d1_exp'] = ''
results['d2_exp'] = ''
results['broker'] = 'Macquarie'
results['top_analyst_long'] = ''
results['top_analyst_short'] = ''

results = results.rename(columns={'Analyst Primary': 'analyst_pri', 'Date of publish': 'date_local', 'Timing': 'release_period',
                                 'Side': 'side', 'Headline Content': 'Headline', 'Summary Content': 'Summary', 'Current Rating': 'rating_curr',
                                 'Current Target Price': 'tp_curr', 'Final Expectancy D0': 'd0_exp'})

results = results[
    ['broker', 'Ticker', 'date_local', 'release_period', 'side', 'Report Type (Broker)', 'Report Type (Broker)', 'gics1',
     'Headline Sentiment', 'Summary Sentiment', 'Headline', 'Summary',
     'rating_curr', 'rating_prev', 
     'tp_curr', 'tp_prev', 'tp_chg_pct', 'up_or_down_side_pct',
      'd0_exp', 'd1_exp', 'd2_exp', 'Blind Long expectancy', 'Blind Short expectancy',
     'top_analyst_long', 'top_analyst_short', 'analyst_pri', 'exch_location', 'exch_location', 'Ticker',
     ]]
DL.toDB(results, f'{DL.database_path}/MAC'+datetime.today().strftime('%m%d')+'.csv')
results

07:18:38 E:/jayyang/OneDrive - Alpha Sherpa Capital/Database/Macquarie\Log/20220527/20220527070835.txt created.
07:18:38 Loading E:/jayyang/OneDrive - Alpha Sherpa Capital/Database/Macquarie/MAC0527.csv
Index(['Ticker', 'Date of publish', 'Timing', 'Side', 'gics1',
       'Report Type (Broker)', 'Headline Sentiment', 'Summary Sentiment',
       'Headline Content', 'Summary Content', 'Current Rating',
       'Current Target Price', 'Final Expectancy D0', 'Analyst Primary',
       'exch_location', 'Blind Long expectancy', 'Blind Short expectancy',
       'Top Analyst (long)', 'Top Analyst (short)', 'Broker'],
      dtype='object')


,broker,Ticker,date_local,release_period,side,Report Type (Broker),Report Type (Broker),gics1,Headline Sentiment,Summary Sentiment,...,d1_exp,d2_exp,Blind Long expectancy,Blind Short expectancy,top_analyst_long,top_analyst_short,analyst_pri,exch_location,exch_location,Ticker
0,Macquarie,6951 JP,2022-05-26,Before,long,TP change,TP change,Health Care Equipment & Services,positive,positive,...,,,0.047209,-0.065481,,,Damian Thong,Japan,Japan,6951 JP
1,Macquarie,3038 JP,2022-05-26,Within,long,TP change,TP change,Food & Staples Retailing,positive,positive,...,,,0.047209,-0.065481,,,Shentao Tang,Japan,Japan,3038 JP
2,Macquarie,7731 JP,2022-05-26,After,long,Ad hoc,Ad hoc,Consumer Durables & Apparel,positive,positive,...,,,0.018730,-0.025160,,,Damian Thong,Japan,Japan,7731 JP
3,Macquarie,6758 JP,2022-05-26,Within,NaN,Ad hoc,Ad hoc,Consumer Durables & Apparel,neutral,positive,...,,,-0.029943,0.015357,,,Damian Thong,Japan,Japan,6758 JP
4,Macquarie,5274 TT,2022-05-26,After,NaN,Rating Upgrade,Rating Upgrade,Semiconductors & Semiconductor Equipment,neutral,positive,...,,,-0.265731,0.201307,,,Jeffrey Ohlweiler,Taiwan,Taiwan,5274 TT
5,Macquarie,9988 HK,2022-05-27,Before,NaN,Earning Review,Earning Review,Retailing,neutral,positive,...,,,0.034494,0.043336,,,Ellie Jiang,Hong Kong,Hong Kong,9988 HK
6,Macquarie,9618 HK,2022-05-27,Before,long,Ad hoc,Ad hoc,Retailing,positive,positive,...,,,0.000320,-0.020964,,,Ellie Jiang,Hong Kong,Hong Kong,9618 HK


In [39]:
results['d0_exp']

0    0.143365
1    0.305467
2    0.088636
3    0.053665
4   -0.113978
5    0.132533
6    0.014642
Name: d0_exp, dtype: float64

In [40]:
results['Headline']

0    JEOL (6951 JP) We see JEOL beating guidance an...
1    Kobe Bussan (3038 JP) Poised to win the race a...
2    Nikon (7731 JP) Much to like in the IR Day; St...
3    Sony (6758 JP) Strategic pivot in games is now...
4    ASPEED Technology (5274 TT) The final piece of...
5     Alibaba Group Holding (BABA US/9988 HK) 4QFY3/22
6    JD.com (JD US/9618 HK) The best pick emerging ...
Name: Headline, dtype: object